# Loading data

In [1]:
import keras

Using TensorFlow backend.


In [ ]:
!wget http://www.manythings.org/anki/fra-eng.zip

In [4]:
!unzip fra-eng.zip

Archive:  fra-eng.zip
  inflating: _about.txt              
  inflating: fra.txt                 


# Preprocessing data

In [3]:
from keras.models import Model
from keras.layers import Input, LSTM, Dense
import numpy as np

batch_size = 64  # Batch size for training.
epochs = 100  # Number of epochs to train for.
latent_dim = 256  # Latent dimensionality of the encoding space.
num_samples = 10000  # Number of samples to train on.

data_path = 'data\\fra.txt'

input_texts = []
target_texts = []
input_characters = set()
target_characters = set()
with open(data_path, 'r', encoding='utf-8') as f:
    lines = f.read().split('\n')
    
print("Total number of translation pair : ", len(lines))

Total number of translation pair :  149862


In [4]:
for line in lines[: min(num_samples, len(lines) - 1)]:
    input_text, target_text = line.split('\t')
    # We use "tab" as the "start sequence" character
    # for the targets, and "\n" as "end sequence" character.
    target_text = '\t' + target_text + '\n'
    input_texts.append(input_text)
    target_texts.append(target_text)
    for char in input_text:
        if char not in input_characters:
            input_characters.add(char)
    for char in target_text:
        if char not in target_characters:
            target_characters.add(char)

In [5]:
print("input characters: ", input_characters)
print("target characters: ", target_characters)

# sample translation
for i in range(5):
  print(input_texts[i]+ "    --->    "+target_texts[i])

input characters:  {'u', 'h', ' ', 't', 'I', '’', 'T', 'Y', 'V', '$', "'", '7', 'i', 'y', '?', ':', '0', '2', '5', 'D', 'Q', 'w', 'p', 'o', 's', 'H', 'O', 'g', ',', 'f', 'k', '1', 'n', 'v', 'd', '!', 'F', '6', '3', 'G', 'j', 'm', 'L', 'r', 'c', 'B', 'e', 'W', 'l', 'b', 'U', 'S', 'a', '8', 'P', '.', 'z', '9', 'N', 'A', 'M', 'q', 'x', '&', 'R', '-', 'K', '4', 'C', 'J', 'E'}
target characters:  {'t', 'V', 'T', 'Y', 'ê', '?', 'ô', '0', '\u202f', 'w', 'o', 'û', ',', 'f', 'n', 'd', '!', '6', 'r', 'c', 'B', 'a', 'Ê', 'P', '.', '-', 'L', 'è', 'u', '’', 'â', 'i', 'î', 'D', 's', 'O', 'Ç', 'k', '3', 'm', 'ï', '\u2009', 'S', '9', ')', 'A', 'M', 'q', 'j', 'À', 'J', 'E', '\xa0', 'É', '«', '(', 'œ', 'é', '5', '‘', 'Q', '\t', '1', 'v', 'à', 'l', 'b', 'U', '8', 'z', 'N', 'ë', '»', 'h', ' ', 'I', 'ç', '\n', '$', "'", 'y', ':', 'p', 'g', 'H', 'F', 'G', 'e', 'x', '&', 'R', 'ù', 'K', 'C'}
Go.    --->    	Va !

Run!    --->    	Cours !

Run!    --->    	Courez !

Wow!    --->    	Ça alors !

Fire!    --->  

In [6]:
input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)
max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])


print('Number of samples:', len(input_texts))
print('Number of unique input tokens:', num_encoder_tokens)
print('Number of unique output tokens:', num_decoder_tokens)
print('Max sequence length for inputs:', max_encoder_seq_length)
print('Max sequence length for outputs:', max_decoder_seq_length)

Number of samples: 10000
Number of unique input tokens: 71
Number of unique output tokens: 94
Max sequence length for inputs: 16
Max sequence length for outputs: 59


### Assign token index

In [7]:
input_token_index = dict(
    [(char, i) for i, char in enumerate(input_characters)])

target_token_index = dict(
    [(char, i) for i, char in enumerate(target_characters)])

### Prepare one-hot encoder data representation : Shape (batch, sentence_len, encode_dim)

In [8]:
encoder_input_data = np.zeros(
    (len(input_texts), max_encoder_seq_length, num_encoder_tokens),
    dtype='float32')
decoder_input_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')
decoder_target_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')

print("encoder input data shape: ", encoder_input_data.shape)
print("decoder input data shape: ", decoder_input_data.shape)
print("decoder target data shape: ", decoder_target_data.shape)

encoder input data shape:  (10000, 16, 71)
decoder input data shape:  (10000, 59, 94)
decoder target data shape:  (10000, 59, 94)


### decoder input :   HELLO    --->   decoder target:  ELLO

In [9]:
for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]] = 1.
    for t, char in enumerate(target_text):
        # decoder_target_data is ahead of decoder_input_data by one timestep
        decoder_input_data[i, t, target_token_index[char]] = 1.
        if t > 0:
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character.
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.


# Build the Sequence to Sequence model

In [10]:
# Define input sequence
encoder_inputs = Input(shape=(None, num_encoder_tokens))
# LSTM as encoder
encoder = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
# we only need 'encoder_outputs'
encoder_states = [state_h, state_c]


# set up the decoder, using `encoder_states` as initial state
decoder_inputs = Input(shape=(None, num_decoder_tokens))
# we set up decoder to return full output sequences, and to return internal
# states as well. we don't use the return states in training model, but
# we will use them in referance
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)

decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

# define a model that will turn 'encoder_input_data' & 'decoder_input_data'
# into 'decoder_target_data'
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [11]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, None, 71)     0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, None, 94)     0                                            
__________________________________________________________________________________________________
lstm_1 (LSTM)                   [(None, 256), (None, 335872      input_1[0][0]                    
__________________________________________________________________________________________________
lstm_2 (LSTM)                   [(None, None, 256),  359424      input_2[0][0]                    
                                                                 lstm_1[0][1]                     
          

In [ ]:
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')
model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
          batch_size=batch_size,
          epochs=epochs,
          validation_split=0.2
         )

model.save('s2s.h5')

Train on 8000 samples, validate on 2000 samples
Epoch 1/100
8000/8000 [==============================] - 15s 2ms/step - loss: 0.9164 - val_loss: 0.9519
Epoch 2/100
8000/8000 [==============================] - 9s 1ms/step - loss: 0.7220 - val_loss: 0.8072
Epoch 3/100
8000/8000 [==============================] - 10s 1ms/step - loss: 0.6183 - val_loss: 0.7037
Epoch 4/100
8000/8000 [==============================] - 10s 1ms/step - loss: 0.5610 - val_loss: 0.6563
Epoch 5/100
8000/8000 [==============================] - 10s 1ms/step - loss: 0.5205 - val_loss: 0.6171
Epoch 6/100
8000/8000 [==============================] - 11s 1ms/step - loss: 0.4895 - val_loss: 0.5988
Epoch 7/100
8000/8000 [==============================] - 10s 1ms/step - loss: 0.4634 - val_loss: 0.5670
Epoch 8/100
8000/8000 [==============================] - 11s 1ms/step - loss: 0.4423 - val_loss: 0.5538
Epoch 9/100
8000/8000 [==============================] - 12s 1ms/step - loss: 0.4225 - val_loss: 0.5410
Epoch 10/100
8000

# Inferance with this model

In [0]:
encoder_model = Model(encoder_inputs, encoder_states)

decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_state_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_lstm(decoder_inputs, initial_state=decoder_state_inputs)

decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)

decoder_model = Model(
    [decoder_inputs]+decoder_state_inputs,
    [decoder_outputs]+decoder_states
)